# 🎓 Notebook 08: Multi-Provider LLM Strategy

Welcome to the final educational module of the RAG Mastery series! In this notebook, we explore the **Provider Abstraction** layer. You will learn how to switch between different AI brains (OpenAI, Gemini, Hugging Face, and Ollama) without changing a single line of your application logic.

### Learning Objectives:
1. Understand the **Adapter Pattern** for LLMs.
2. Learn how to configure multiple providers in RAG Engine Mini.
3. Compare response formats and performance across backends.

## 1. Setup and Imports

We will use the project's dependency injection container to see how the system swaps implementations.

In [ ]:
import os
import asyncio
from dotenv import load_dotenv
from src.core.bootstrap import get_container, reset_container
from src.core.config import get_settings

load_dotenv() # Ensure you have your keys in .env

## 2. Exploring the Adapters

Let's look at how the same query is handled by different backends using the `LLMPort` interface.

In [ ]:
async def test_llm(backend_name: str):
    print(f"--- Testing Backend: {backend_name} ---")
    
    # Temporarily override settings
    os.environ["LLM_BACKEND"] = backend_name
    reset_container()
    
    container = get_container()
    llm = container["llm"]
    
    prompt = "What are the three pillars of a robust RAG system? Answer concisely."
    
    try:
        response = await llm.generate(prompt)
        print(f"Response: {response}\n")
    except Exception as e:
        print(f"Error with {backend_name}: {e}\n")

# Note: These require valid API keys in your .env
# await test_llm("openai")
# await test_llm("gemini")
# await test_llm("huggingface")
print("Uncomment the lines above to run live tests with your keys!")

## 3. The Power of `generate_stream`

A premium user experience requires streaming. Our abstraction ensures that regardless of the provider's unique streaming implementation (Server-Sent Events, GRPC, etc.), the application receives a simple `AsyncGenerator`.

In [ ]:
async def demo_streaming(backend_name: str):
    os.environ["LLM_BACKEND"] = backend_name
    reset_container()
    llm = get_container()["llm"]
    
    print(f"Streaming from {backend_name}...")
    async for chunk in llm.generate_stream("Tell me a 2-sentence story about a robot engineer."):
        print(chunk, end="", flush=True)
    print("\n")

# await demo_streaming("openai")

## 4. Why This Matters: Architectural Freedom

By isolating the LLM logic into adapters:
1. **Maintenance**: If Hugging Face changes their API tomorrow, you only fix one file (`huggingface_llm.py`).
2. **Cost Control**: You can use cheaper models for retrieval grading and expensive models for the final answer.
3. **Local-First**: You can develop offline using **Ollama** and deploy to **Gemini/OpenAI** for production.

--- 
**Congratulations!** You have completed the RAG Engineering Mastery series. You now possess the knowledge to build, scale, and optimize production-grade search systems.